In [32]:
import networkx as nx
import math
import scipy.io as sio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import det
from numpy import cov

In [79]:
data = sio.loadmat('Market.mat')['DI']

In [61]:
#the nodes are the companies:
vertex_set = [i for i in range(12)]
G = nx.Graph()
for v in vertex_set:
    G.add_node(v)

In [62]:
all_nodes = set(vertex_set)

In [82]:
all_nodes

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

In [80]:
print(data.shape)

(7, 190, 12)


In [83]:
for i in vertex_set:
    for j in vertex_set:
        if i == j:
            continue
        else:
            s_x = {i}
            s_y = {j}
            s_z = all_nodes - s_x - s_y
            x_t = data[:,:,i]  #company i
            y_t = data[:,:,j]  #company j

            z_t = [] #remaining companies
            for k in s_z:
                z_t.append(data[:,:,k])
                
            z_t = np.asarray(z_t)    

            print(y_t.shape,z_t.shape)
            
            #for t in np.arange(0,7)+1:
                
             #   print(y_t[:t,:].shape,z_t[:t,:,:].shape)
                #m1 = np.concatenate()


(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (10, 7, 190)
(7, 190) (

In [ ]:
nx.draw(G, with_labels = True)